In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# yes24 > 한국 에세이 2015발표작 이후로 한정 > 24개씩 1209번 페이지까지 스크래이핑
response = requests.get('https://www.yes24.com/Product/Category/Display/001001047001')

html = response.text
soup = BeautifulSoup(html, 'html.parser')


In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import json
import time
import re


driver = webdriver.Chrome()
driver.get('https://www.yes24.com/Product/Category/Display/001001047001')
time.sleep(1)

main_scroll_cnts = 5
sub_scroll_cnts = 3


#1 스크롤을 내리고 - n번째 페이지 방문
    #2 그 페이지 각각의 도서를 순회하며
        #3 책소개/ 책속으로 text를 수집-저장
